# Writing like Shakespeare
Use LSTM & Keras
- Obtain local input data
- Prepare training data
- Load the trained model
- Train the uploaded model with new input data (just one epoch)
- Predict with newly trained model

In [1]:
### Import modules and functions
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
### Load text data from local drive
text = io.open('data/shakespeare.txt', encoding='utf-8').read().lower()

In [3]:
### Prepare data
Tx = 40  #Sequence length for each training exmple
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
#print('number of unique characters in the corpus:', len(chars))

In [4]:
### Create Training data
X = [] # list of sentence of 40 characters
Y = [] # next charater of each sentence
   
for i in range(0, len(text) - Tx, 3):
    X.append(text[i: i + Tx])
    Y.append(text[i + Tx])

print('number of training examples:', len(X))

number of training examples: 31412


In [5]:
### Vectorization of Training data 
m = len(X)
n_x = len(chars)
x = np.zeros((m, Tx, n_x), dtype=np.bool) # contains False or True
y = np.zeros((m, n_x), dtype=np.bool) # contains False or True
for i, sentence in enumerate(X):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[Y[i]]] = 1
        

In [6]:
### Load Model 
model = load_model('models/model_shakespeare_kiank_350_epoch.h5')

In [7]:
model.summary

<bound method Container.summary of <keras.engine.training.Model object at 0x000001A46B1CA2E8>>

In [ ]:
### Train model just one more epoch with the current data 
model.fit(x, y, batch_size=128, epochs=1)

Epoch 1/1
31412/31412 [==============================] - 128s 4ms/step - loss: 2.5519


In [ ]:
### Obtain input
generated = ''

usr_input = input("Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: ")
# zero pad the sentence to Tx characters.
sents = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
generated += usr_input 

In [ ]:
### Print predicted text from input
sys.stdout.write("\n\nHere is your poem: \n\n") 
sys.stdout.write(usr_input)
for i in range(400):

    ## Prepare input data
    x_pred = np.zeros((1, Tx, len(chars))) #input characters
    for t, char in enumerate(sents):
        if char != '0':
            x_pred[0, t, char_indices[char]] = 1.

    ## Predict with the trained model and input data
    preds = model.predict(x_pred, verbose=0)[0]
    
    # helper function to sample an index from a probability array
    temperature=1.0
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1) # distribute "preds" propotionally so sum(preds)= 1
    # pick next char based on prob
    next_index = np.random.choice(range(len(chars)), p = probas.ravel())n 
    next_char = indices_char[next_index] # character 

    generated += next_char
    sents = sents[1:] + next_char # add next character in the current sentence - 1 character (always 40 long)

    ## printout next character
    sys.stdout.write(next_char) 
    sys.stdout.flush()
    if next_char == '\n':
        continue